In [ ]:
import pulp
import datetime

# --- 1. PARAMETER ---
N_units = 27
N_tasks_per_unit = 12
Capacity = 2

Durations = {
    "P1": 1, "P3": 1, "P6": 1, "P12": 2
}

Task_Type_Sequence = [
    "P1", "P1", "P3", "P1", "P1", "P6", "P1", "P1", "P3", "P1", "P1", "P12"
]

Max_Days = 365

start_calendar_date = datetime.date(2025, 1, 1) # Rabu
Working_day = {}
for d in range(1, Max_Days + 1):
    current_date = start_calendar_date + datetime.timedelta(days=d-1)
    Working_day[d] = current_date.weekday() < 5

Month_Day_Ranges = {}
current_month_start_date = start_calendar_date
current_day_in_year = 1
for month_num in range(1, 13):
    current_month_start_date_actual = datetime.date(2025, month_num, 1)
    start_day_of_month = (current_month_start_date_actual - start_calendar_date).days + 1
    if month_num < 12:
        next_month_first_day = datetime.date(2025, month_num + 1, 1)
        end_day_of_month = (next_month_first_day - start_calendar_date).days
    else:
        end_day_of_month = Max_Days
    Month_Day_Ranges[month_num] = (start_day_of_month, end_day_of_month)

# --- 2. MODEL INITIATION ---
model = pulp.LpProblem("Maintenance_Scheduling_Optimization", pulp.LpMinimize)

# --- 3. VARIABEL KEPUTUSAN ---
Start_day = pulp.LpVariable.dicts("Start_day",
                             ((u, t) for u in range(1, N_units + 1) for t in range(1, N_tasks_per_unit + 1)),
                             lowBound=1, cat='Integer')

End_day = pulp.LpVariable.dicts("End_day",
                           ((u, t) for u in range(1, N_units + 1) for t in range(1, N_tasks_per_unit + 1)),
                           lowBound=1, cat='Integer')

Is_task_running_on_day = pulp.LpVariable.dicts("Is_task_running_on_day",
                                          ((u, t, d) for u in range(1, N_units + 1)
                                           for t in range(1, N_tasks_per_unit + 1)
                                           for d in range(1, Max_Days + 1)), cat='Binary')

# --- 4. OBJECTIVE FUNCTION ---
Overall_Completion_Day = pulp.LpVariable("Overall_Completion_Day", lowBound=1, cat='Integer')
model += Overall_Completion_Day, "Minimize_Overall_Completion_Day"

for u in range(1, N_units + 1):
    model += Overall_Completion_Day >= End_day[u, N_tasks_per_unit], f"Completion_Constraint_Unit_{u}"


# --- 5. CONSTRAINTS ---

# 5.1. Kendala Durasi Tugas dan Hubungan Start_day / End_day
for u in range(1, N_units + 1):
    for t in range(1, N_tasks_per_unit + 1):
        task_type = Task_Type_Sequence[t-1]
        duration = Durations[task_type]
        model += End_day[u, t] == Start_day[u, t] + duration - 1, f"Duration_Link_U{u}_T{t}"


# 5.2. Kendala Urutan Perawatan (Precedence Constraints)
for u in range(1, N_units + 1):
    for t in range(1, N_tasks_per_unit):
        model += Start_day[u, t+1] >= End_day[u, t] + 1, f"Precedence_Unit_{u}_Task_{t}"


# 5.3. Kendala Kapasitas Fasilitas & Linking Is_task_running_on_day
M = Max_Days + 1 # Use Max_Days + 1 for robustness.

for u in range(1, N_units + 1):
    for t in range(1, N_tasks_per_unit + 1):
        for d in range(1, Max_Days + 1):
            # Kendala 1: Jika Is_task_running_on_day[u,t,d] = 1, maka d >= Start_day[u,t]
            model += Start_day[u,t] - d <= M * (1 - Is_task_running_on_day[u,t,d]), \
                     f"Link_StartBound_U{u}_T{t}_D{d}_1"

            # Kendala 2: Jika Is_task_running_on_day[u,t,d] = 1, maka d <= End_day[u,t]
            model += d - End_day[u,t] <= M * (1 - Is_task_running_on_day[u,t,d]), \
                     f"Link_EndBound_U{u}_T{t}_D{d}_2"

            # Kendala 3: Jika d berada dalam interval [Start_day[u,t], End_day[u,t]],
            # maka Is_task_running_on_day[u,t,d] harus dipaksa menjadi 1.
            # Ini adalah formulasi yang lebih sering saya gunakan dan lebih stabil.
            # Ini memaksa Is_task_running_on_day menjadi 1 jika d di dalam range,
            # dan menjadi 0 jika d di luar range.
            model += Start_day[u,t] - d + Is_task_running_on_day[u,t,d] * M >= 1, f"Link_ForceActive_U{u}_T{t}_D{d}_3a"
            model += d - End_day[u,t] + Is_task_running_on_day[u,t,d] * M >= 1, f"Link_ForceActive_U{u}_T{t}_D{d}_3b"
            model += Start_day[u,t] - d + End_day[u,t] - d + 1 <= Is_task_running_on_day[u,t,d] * M + (1 - Is_task_running_on_day[u,t,d]) * (M - Durations[Task_Type_Sequence[t-1]]), f"Link_ForceActive_U{u}_T{t}_D{d}_3c"


# Kendala Kapasitas Harian: Jumlah semua tugas yang berjalan pada hari tertentu
# tidak boleh melebihi kapasitas yang tersedia (2 unit).
for d in range(1, Max_Days + 1):
    model += pulp.lpSum(Is_task_running_on_day[u, t, d] for u in range(1, N_units + 1)
                                                    for t in range(1, N_tasks_per_unit + 1)) <= Capacity, \
             f"Daily_Capacity_D{d}"


# 5.4. Kendala Hari Kerja
# Perawatan hanya dapat dilakukan pada hari kerja (Senin-Jumat).
# Jika Working_day[d] adalah False (hari libur), maka Is_task_running_on_day[u,t,d] harus 0.
for u in range(1, N_units + 1):
    for t in range(1, N_tasks_per_unit + 1):
        for d in range(1, Max_Days + 1):
            if not Working_day[d]:
                model += Is_task_running_on_day[u,t,d] == 0, f"No_Work_On_NonWorkingDay_U{u}_T{t}_D{d}"


# 5.5. Kendala Siklus Tahunan per Unit
for u in range(1, N_units + 1):
    model += End_day[u, N_tasks_per_unit] <= Start_day[u, 1] + Max_Days -1, f"Annual_Cycle_Unit_{u}"
    model += Start_day[u, 1] <= Max_Days, f"Start_Day_Limit_U{u}"


# 5.6. KENDALA BARU: Perawatan Terikat pada Bulan Spesifik
for u in range(1, N_units + 1):
    for t in range(1, N_tasks_per_unit + 1):
        target_month_num = t # Tugas ke-1 di bulan 1, tugas ke-2 di bulan 2, dst.
        
        if target_month_num not in Month_Day_Ranges:
            raise ValueError(f"Target month {target_month_num} not found in Month_Day_Ranges.")
            
        month_start_day, month_end_day = Month_Day_Ranges[target_month_num]

        model += Start_day[u,t] >= month_start_day, \
                 f"Month_Constraint_Start_U{u}_T{t}"
        
        model += End_day[u,t] <= month_end_day, \
                 f"Month_Constraint_End_U{u}_T{t}"


# --- 6. SOLVE THE MODEL ---
print("Memulai proses optimasi...")
# Batas waktu komputasi akan menjadi sangat penting di sini.
# Model ini akan sangat sulit dipecahkan. Sangat mungkin akan berakhir sebagai infeasible atau Not Solved.
model.solve(pulp.PULP_CBC_CMD(msg=1, timeLimit=3600)) # Diberi batas waktu 60 menit (3600 detik)


# --- 7. TAMPILKAN HASIL ---
print(f"\nStatus Solusi: {pulp.LpStatus[model.status]}")

if model.status == 1: # Optimal
    print(f"Hari Penyelesaian Keseluruhan Optimal: {int(pulp.value(Overall_Completion_Day))}")
    print("\nJadwal Mulai/Selesai Perawatan per Unit dan Tugas (untuk beberapa unit pertama):")
    for u in range(1, min(N_units + 1, 5)):
        print(f"\n--- Unit {u} ---")
        for t in range(1, N_tasks_per_unit + 1):
            start_day_val = pulp.value(Start_day[u, t])
            end_day_val = pulp.value(End_day[u, t])
            task_type = Task_Type_Sequence[t-1]
            if start_day_val is not None and end_day_val is not None:
                start_date_actual = start_calendar_date + datetime.timedelta(days=int(start_day_val) - 1)
                end_date_actual = start_calendar_date + datetime.timedelta(days=int(end_day_val) - 1)
                print(f"  Tugas {t} ({task_type}): Mulai = {start_date_actual.strftime('%Y-%m-%d')} (Hari {int(start_day_val)}), Selesai = {end_date_actual.strftime('%Y-%m-%d')} (Hari {int(end_day_val)})")
            else:
                print(f"  Tugas {t} ({task_type}): Tidak ada solusi ditemukan untuk tugas ini.")

    print("\nVerifikasi Pemanfaatan Kapasitas Harian (Contoh 30 hari pertama):")
    for d in range(1, min(30, Max_Days) + 1):
        active_tasks_on_day_list = []
        for u in range(1, N_units + 1):
            for t in range(1, N_tasks_per_unit + 1):
                if pulp.value(Is_task_running_on_day[u,t,d]) is not None and pulp.value(Is_task_running_on_day[u,t,d]) > 0.5:
                    active_tasks_on_day_list.append(f"U{u}T{t}")
        current_date_display = start_calendar_date + datetime.timedelta(days=d-1)
        if active_tasks_on_day_list:
            print(f"Hari {d} ({current_date_display.strftime('%Y-%m-%d, %a')}): {len(active_tasks_on_day_list)} tugas aktif: {', '.join(active_tasks_on_day_list)}")
        else:
            print(f"Hari {d} ({current_date_display.strftime('%Y-%m-%d, %a')}): 0 tugas aktif.")


elif model.status == 0: # Not Solved (misal: karena timeLimit tercapai)
    print("\nModel tidak menemukan solusi optimal dalam batas waktu yang ditentukan.")
    if pulp.value(Overall_Completion_Day) is not None:
        print(f"Nilai Overall Completion Day terbaik yang ditemukan sejauh ini: {int(pulp.value(Overall_Completion_Day))}")
    print("Mungkin ada solusi yang lebih baik jika waktu komputasi ditambah atau menggunakan solver yang lebih powerful.")

else: # Termasuk Infeasible (-1), Unbounded (-2), dll.
    print(f"\nModel tidak memiliki solusi atau tidak dapat diselesaikan: {pulp.LpStatus[model.status]}.")
    print("Sangat mungkin model ini INFEASIBLE karena kendala yang terlalu ketat.")
    print("Coba batasi jumlah unit atau kapasitas untuk debug.")